# ДЗ1. CLAP. Обучение проекции из аудио в текстовое пространство CLIP

### Описание задания

В этом задании вы построите упрощённый вариант модели CLAP (Contrastive Language-Audio Pretraining):

- аудио прогоняется через предобученный аудио-энкодер (например, `LanguageBindAudio`, `CNN14/16` или другой);
- текстовое описание пропускается через предобученный текстовый энкодер CLIP;
- поверх аудио-векторов обучается линейный адаптер, который отображает аудио в то же пространство, что и текстовые эмбеддинги CLIP;
- обучение идёт по *контрастивному лоссу*, все энкодеры заморожены, обучаются только параметры аудио-проекции (и, при желании, температура в лоссе);
- качество полученного аудио-текстового пространства оценивается на задаче классификации / retrieval аудио по текстам на `AudioCaps`.

Идея оценки: если всё сделано правильно, для аудио и его описания косинусное сходство эмбеддингов будет выше, чем для аудио и нерелевантных текстов.


**Формулировка задач**

0. Выбор аудио-энкодера.
   Выберите и обоснуйте предобученный аудио-энбеддер:  
   - `LanguageBindAudio`,  
   - или CNN-модель (например, PANNs CNN14/16),  
   - или другой открытый аудио-энкодер, который выдаёт фиксированный эмбеддинг.

1. Подсчёт эмбеддингов.
   - Посчитайте аудио-векторы для всех аудио из `AudioCaps` с помощью выбранного энкодера.  
   - Посчитайте текстовые векторы для подписей с помощью `CLIP text encoder`.

2. Линейная аудио-проекция.
   - Реализуйте модель `AudioProjection`, переводящую аудио-эмбеддинг в размерность текстового эмбеддинга CLIP.

3. Контрастивное обучение.
   - Обучите аудио-проекцию на датасете `AudioCaps` по схеме аудио ↔ текст с контрастивным лоссом.  
   - Аудио-энкодер и CLIP должны быть полностью заморожены.

4. Оценка качества.
   - Оцените качество полученного аудио-текстового пространства на задаче классификации/ретривала аудио:  
     для каждого аудио найдите наиболее похожую текстовую подпись в батче/валидации и посчитайте `accuracy@1/3/10`.  
   - Сравните результаты с *случайным бейзлайном*.


### Сеттинг

> Подготовьте все необходимые импорты и загрузите необходимые данные.

In [11]:
# Для загрузки AudioCaps можно воспользоваться этим кодом
import os

# !gdown --id 1FAVKNWXp5afgoNmclDwnj8j_OFTBRmIb -O audiocaps.zip
# !unzip audiocaps -d audiocaps

DATA_ROOT = "data/audiocaps"
print("Files in DATA_ROOT:", os.listdir(DATA_ROOT))

Files in DATA_ROOT: ['audiocaps_val_new.tsv', 'audiocaps_val.tsv', 'test_texts.json', 'val_texts.json', 'audiocaps_test_new.tsv', 'audiocaps_train.tsv', 'audiocaps_test.tsv', 'audio', 'audio_embeddings_train.pkl', 'train_processed.pkl']


### Задание 1. Подготовка аудио- и текстовых энкодеров (2 балла)

В этом задании вам нужно:

1. Выбрать аудио-энкодер и инициализировать его.
2. Инициализировать текстовый энкодер CLIP. Вы свободны выбирать самостоятельно, какой имеено.
3. Заморозить параметры обоих энкодеров (мы не дообучаем их, а учим только линейный адаптер).

Вы можете:

* использовать `LanguageBindAudio` (потребует установки репозитория и зависимостей);
* или подставить свою аудио-модель (главное - чтобы на выходе был вектор фиксированной размерности).


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from typing import Dict, List, Optional, Tuple

import sys
import numpy as np
import torch
from torch.utils.data import Dataset
import pandas as pd
from tqdm.auto import tqdm

from src.model import AudioEncoder, EncoderConfig, TextEncoder

config = EncoderConfig(
    audio_encoder_name="laion/clap-htsat-unfused",
    text_encoder_name="openai/clip-vit-base-patch32",
)

print("\nConfig:")
print(config)

audio_encoder = AudioEncoder(config.audio_encoder_name)
text_encoder = TextEncoder(config.text_encoder_name)

/home/cloexec/Desktop/HSE/multimodal/HW1/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Config:
EncoderConfig(audio_encoder_name='laion/clap-htsat-unfused', text_encoder_name='openai/clip-vit-base-patch32', audio_sample_rate=48000, batch_size=32, checkpoint_freq=100)


### Задание 2. Предподсчёт аудио- и текстовых эмбеддингов (3 балла)

> Важный момент, который пригодится вам и в других домашних.

Чтобы не тратить время на многократный прогон энкодеров при обучении, следует:

1. Предварительно посчитывать аудио-эмбеддинги для каждого `.flac` в train/val/test.
2. Записывать их в файл формата `pickle` (например), где ключ - имя файла, значение - numpy-вектор.
3. Аналогично посчитать текстовые эмбеддинги для подписей через CLIP и совместить их с аудио.

Рекомендуемая структура:

* функция `extract_audio_vectors_with_checkpointing(...)` - обходит файлы, считает эмбеддинги, периодически делает чекпоинты;
* функция `extract_text_embeddings(texts, clip_model, clip_processor)` - возвращает список текстовых эмбеддингов;
* функция `process_dataset(...)` - читает `.tsv`, мержит аудио-эмбеддинги и текстовые, сохраняет список словарей вида  
  `{"uniq_id": ..., "audio_embedding": ..., "text_embedding": ...}` в pickle.

> Вы вольны отходить от предлагаемой структуры.

In [3]:
import pickle
from pathlib import Path

from src.dataset import extract_audio_embeddings_with_checkpointing, process_dataset, AudioTextDataset, get_embedding_dimensions

DATA_ROOT = "data/audiocaps/"
CHECKPOINT_FREQ = 50
BATCH_SIZE = 32
CHECKPOINT_FILENAME = "audio_embeddings_train.pkl"
PROCESSED_DATASET_FILENAME = "train_processed.pkl"

data_root = Path(DATA_ROOT)
assert data_root.exists(), "No data directory found"

audio_dir = data_root / "audio" / "train"
assert audio_dir.exists()

assert audio_dir.exists() and list(audio_dir.glob("*.flac")), "No audio files found in train directory"

tsv_path = data_root / "audiocaps_train.tsv"
assert tsv_path.exists(), "No train.tsv file found"

print(f"Extracting Audio Embeddings (with checkpointing) from {audio_dir}...")

checkpoint_path = data_root / CHECKPOINT_FILENAME

if not checkpoint_path.exists():
    audio_embeddings = extract_audio_embeddings_with_checkpointing(
        audio_dir=audio_dir,
        encoder=audio_encoder,
        checkpoint_path=checkpoint_path,
        checkpoint_freq=CHECKPOINT_FREQ,
        resume=True
    )
    print(f"Extracted {len(audio_embeddings)} audio embeddings")
else:
    audio_embeddings = pickle.load(open(checkpoint_path, 'rb'))
    print(f"Loaded {len(audio_embeddings)} audio embeddings from checkpoint {checkpoint_path}")

print("Processing Dataset (merge audio + text)...")

dataset = process_dataset(
    tsv_path=tsv_path,
    audio_embeddings=audio_embeddings,
    text_encoder=text_encoder,
    output_path=data_root / PROCESSED_DATASET_FILENAME,
    batch_size=BATCH_SIZE
)

print(f"Processed dataset with {len(dataset)} samples")

Extracting Audio Embeddings (with checkpointing) from data/audiocaps/audio/train...
Loaded 49515 audio embeddings from checkpoint data/audiocaps/audio_embeddings_train.pkl
Processing Dataset (merge audio + text)...
Processing 49490 samples from audiocaps_train.tsv


Processing batches: 100%|███████████████████████████████████████████████████████████████████████| 1547/1547 [00:38<00:00, 40.20it/s]


Saved 49490 samples to data/audiocaps/train_processed.pkl
Processed dataset with 49490 samples


### Задание 3. Линейный аудио-адаптер и контрастивный лосс (3 балла)


Теперь, когда у нас есть пары *audio_embedding, text_embedding*, реализуем:

1. Класс `AudioTextDataset`, который читает pickle с комбинированными эмбеддингами.
2. Линейную модель `AudioProjection`, переводящую аудио-эмбеддинг в размерность текстового.
3. Контрастивный лосс для аудио↔текст:
   - нормализовать эмбеддинги по L2;
   - посчитать матрицу сходства;
   - задать таргеты как `targets = arange(batch_size)`;
   - вычислить `CrossEntropyLoss` как для строк audio→text и для строк text→audio, усреднить.

Обучаем **только** `AudioProjection` (и, по желанию, параметр temperature).


In [9]:
print("Creating PyTorch Dataset...")
pytorch_dataset = AudioTextDataset(data_root / PROCESSED_DATASET_FILENAME)

print("Great success")

Creating PyTorch Dataset...
Loaded dataset with 49490 samples
Great success


In [8]:
audio_emb, text_emb = pytorch_dataset[0]
print("Sample:")
print(f"\taudio embedding shape: {audio_emb.shape}")
print(f"\ttext embedding shape: {text_emb.shape}")

Sample:
	audio embedding shape: torch.Size([512])
	text embedding shape: torch.Size([512])


### Задание 4. Оценка качества на задаче классификации аудио (2 балла)


Теперь нужно понять, насколько хорошо аудио-векторы после проекции "попадают" в пространство текстовых эмбеддингов:

1. Посчитайте проекции аудио для всех примеров в валидации.
2. Для каждого аудио найдите `top-k` наиболее похожих текстов по косинусному сходству (или скалярному произведению после L2-нормализации).
3. Посчитайте `accuracy@1`, `accuracy@3`, `accuracy@10`, т.е. долю случаев, когда "правильный" текст попал в топ-k.
4. Сравните с неким *случайным бейзлайном*: для каждого аудио выберите `k` случайных текстов и посчитайте такую же метрику.

> Важно: в батче класс "правильного" текста для i-го аудио - это индекс i (как в контрастивном лоссе).

In [3]:
raise NotImplementedError
# TODO

### Вывод

Оформите, пожалуйста, небольшой вывод. Например, можно воспрользоваться следующим планом:

   * какую аудио-модель вы выбрали и почему;
   * как вели себя потери на обучении;
   * какие значения метрик получились и насколько они превосходят случайный baseline;
   * любые наблюдения (например, зависимость от числа эпох, размера батча и т.д.);
   * милые пожелания ассистенту/лектору, который будет это проверять.

your text here TODO